# 評価指標の最適化の例

## balanced accuracyの最適化

$$
Balanced Accuracy = \frac{1}{M}\sum_{m=1}^{M}\frac{r_m}{n_m}
$$

$M:$クラス数

$n_m:$クラス$m$に属するデータ数

$r_m:$クラス$m$に属するデータの内、正しく予測できたデータ数

通常のaccuracyを真の値のクラスの割合の逆数に比例した重み付けで計算したもの。

少数しか無いクラスの正しい予測がスコアに大きな影響をもたらす形になる。（少数クラスの正答のほうが価値がある。）

そのため不均衡データセットによる分類の場合、クラス別の予測確率×クラスの割合の逆数が最大になるクラスを選択する方法や、クラス別の重み付けを考慮する必要がある。

In [1]:
## クラス別重み付け

import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
from sklearn.metrics import balanced_accuracy_score

# 不均衡なデータセットにする
data = load_wine().data[:139]
target = load_wine().target[:139]
print(pd.Series(target).value_counts(sort=False))
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.2, random_state=42, stratify=target)

0    59
1    71
2     9
dtype: int64


In [2]:
# class weightを考慮しなければ精度は低い
model = SVC(random_state=42)
model.fit(train_x, train_y)
pred = model.predict(valid_x)

print('balanced accuracy score: ', balanced_accuracy_score(valid_y, pred))

balanced accuracy score:  0.6388888888888888


In [3]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', np.unique(target), target)
print('class weights: ', class_weights)

# class_weight='balanced'でも同じ効果が得られる
model = SVC(random_state=42, class_weight={0: class_weights[0], 1: class_weights[1], 2: class_weights[2]})
model.fit(train_x, train_y)
pred = model.predict(valid_x)

print('balanced accuracy score: ', balanced_accuracy_score(valid_y, pred))

class weights:  [0.78531073 0.65258216 5.14814815]
balanced accuracy score:  0.8293650793650794
